### 存储参数的地方

Tensor对象。Variable对象是对Tensor对象的再包装。API几乎和Tensor的API一致。   
  
    
### tensor对象的继承体系    

  
在python中定义了Parameter类  
```python
class Parameter(torch.Tensor)
```

在python中定义了torch.Tensor类  
```python
class Tensor(torch._C._TensorBase)
```

在C++中定义了Variable类  （求微分）
```c++
 struct TORCH_API Variable : public at::Tensor
 PyObject* Py_InitModule(char *name, PyMethodDef *methods)  
```

创建torch._C  
```c++
 Py_InitModule("torch._C", methods.data()）
```

创建 torch._C._TensorBase  
```c++
PyModule_AddObject(module, "_TensorBase",   (PyObject *)&THPVariableType);  
  
```  
  
<br>  

### Tensor对象    


  
$\color{red}{往\_C模块里添加 \_ TensorBase成员：}$     
  
添加成员（调用要添加成员模块的初始化函数。这些成员模块的初始化函数的参数，是其父类的模型的引用）。
```C++  
 #例如往_ C模块里添加 _ TensorBase 模块。 
 # 调用 _ TensorBase 模块 的 初始化函数。并传入_ C模块的引用
 #  _ TensorBase的文件名是python_variabl.h 要在_C的文件里引用。
  ASSERT_TRUE(THPVariable_initModule(module));
```


以下是_ C模块里被添加模块的文件。（被添加模块也要按照python对C扩展规范编写。） _ TensorBase的文件名是python_variabl.cpp  
  
模块名叫 _ TensorBase ， 文件名 python_variabl.cpp
  
```C++
# _ TensorBase 结构体 ，name 和 方法 数组  等一系列变量

PyTypeObject THPVariableType = {
  PyVarObject_HEAD_INIT(nullptr, 0)
  "torch._C._TensorBase",                      /* tp_name */
  
      ……
  nullptr,                                     /* tp_methods */
 
  
        ……    

};  
  
#_ TensorBase的初始化，参数是_C模块的引用
# vector<PyMethodDef> methods  把_ TensorBase的函数加入犯法数组  
# PyModule_AddObject(module, "_TensorBase",   (PyObject *)&THPVariableType);  把THPVariableType类型在_C里注册为_TensorBase  
#  第一个参数是父类，第二个参数是别名，第三个参数是要加入的类型。
# 同时这个初始化里还会为初始化initTorchFunctions，initTensorImplConversion这两个函数
bool THPVariable_initModule(PyObject *module)
{
  static std::vector<PyMethodDef> methods;
  THPUtils_addPyMethodDefs(methods, torch::autograd::variable_methods);
  THPUtils_addPyMethodDefs(methods, extra_methods);
  THPVariableType.tp_methods = methods.data();
  if (PyType_Ready(&THPVariableType) < 0)
    return false;
  Py_INCREF(&THPVariableType);
  PyModule_AddObject(module, "_TensorBase",   (PyObject *)&THPVariableType);
  torch::autograd::initTorchFunctions(module);
  torch::autograd::initTensorImplConversion(module);
  return true;
}

```

### Tensor 在C++中的继承体系  
  
  Tensor比较重要的特征有：tensor的维度信息，tensor的值内容，tensor的梯度grad，tensor的type ， tensor的backend等。一个tensor需要精巧的内存管理  
    
  一个tensor是由DataPtr、Storagelmp、Storage、Tensorlmp、Tensor、Variable::lmp、Variable、Variable::AutogradMeta这些底层类组成的。  
    
  1、DataPtr位于最底层，用来直接维护tensor所需内存  
    
  2 、Storagelmp、Storage 管理侵入式引用指针  
    
  3、Tensorlmp、Tensor  
    
 4、Variable::lmp、Variable  动态图；Variable ：有自动微分系统的张量  
   
 5、Variable::AutogradMeta  ；所有和autograd相关的东西都定义在这个类中。  
   
   
 <br>  
   
 ### 如何创建一个Tensor  
   
 ```Python 

  #第一种方法使用torch方法 ,默认,requires_grad=True 
  TensorData1=torch.empty(7,19 ,requires_grad=False)
  TensorData2=torch.rand(7,19,requires_grad=False)
  TensorData3=torch.zeros(7,19,dtype=torch.long,requires_grad=False)
  TensorData4=torch.tensor(7,19,requires_grad=False)  
    
  #TensorData4.require_grad_(True)

  # 第二种方法 实例化一个Tensor对象 ，,默认,requires_grad=True   
  TensorData5=torch.Tensor([1,2],,requires_grad=True)

```  
  
### Tensor 的 backward()  
  
  对Tensor内保存的动态图进行反向传播计算梯度。